# Preparación y curación de los datos

---
## Convenciones

### Dataframes principales

`msg_df` dataframe de mensajes

`md_df` dataframe de metadata

`agg_msg_df` dataframe de mensajes totalizado

`res_df` dataframe resultante del merge del dataframe de metadata y el dataframe de mensajes totalizado

### Prefijos de columnas nuevas

`n_` cantidad count

`n_words` cantidad de palabras

`avg_` promedio

---


## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from ast import literal_eval
from collections import Counter

import re

from type_to_fix import T2F

## Columnas de la metadata a incluir en el resultado

**Nota:**
*Incluidas en la notebook para que sean facilmente "comentables"*

In [ ]:
C2K = list()
C2K.append('session_id')
C2K.append('tutor_id')
C2K.append('student_id')
C2K.append('timestamp')
C2K.append('feedback_score')
C2K.append('wait_time')
C2K.append('ended_by_reason')
C2K.append('session_category')
C2K.append('consolidated_session_category')
C2K.append('student_complained')
C2K.append('student_complaint_clarity')
C2K.append('student_complaint_speed')
C2K.append('student_complaint_subject')
C2K.append('student_complaint_other')
C2K.append('session_tag_cheating')
C2K.append('session_tag_inappropriate')
C2K.append('session_tag_other_subject')
C2K.append('session_tag_no_material')
C2K.append('session_tag_student_left')
C2K.append('session_tag_student_not_engaging')
C2K.append('session_tag_used_whiteboard')
C2K.append('student_rating')
C2K.append('length_of_session')
C2K.append('avg_tutor_response_time')
C2K.append('max_tutor_response_time')
C2K.append('subject')
C2K.append('subtopic')
# C2K.append('rubric_version')
C2K.append('tutor_gender')
C2K.append('tutor_international_name')
C2K.append('tutor_fired')
C2K.append('tutor_math_exam_score')
C2K.append('tutor_physics_exam_score')
C2K.append('tutor_chemistry_exam_score')
C2K.append('tutor_last_sign_in_country')
C2K.append('tutor_age')

## Carga de archivos CSV de origen

**Nota:**
*Descomentar el archivo de mensajes necesario*

In [ ]:
data_dir = os.path.join('..', 'dataset')

# dataset de mensajes
msg_data_file_name = 'dev_yup_messages_preprocessed.csv'
#msg_data_file_name = 'train_yup_messages_preprocessed.csv'
#msg_data_file_name = 'test_yup_messages_preprocessed.csv'
#msg_data_file_name = 'yup_messages_preprocessed.csv'

msg_full_data_file_name = os.path.join(data_dir, msg_data_file_name)
msg_df = pd.read_csv(msg_full_data_file_name)
msg_df = msg_df.astype({'created_at': 'datetime64[ns, UTC]'})

# dataset de metadata
md_data_file_name = 'datadump-20150801-20171219.csv'
md_full_data_file_name = os.path.join(data_dir, md_data_file_name)
md_df = pd.read_csv(md_full_data_file_name, low_memory=False)

# dataset de metadata
md_df.columns = [column.strip() for column in md_df.columns]
md_df = md_df[C2K]
md_df = md_df.astype(T2F)

# Caracterización del dataset de mensajes
print ('\nDimensiones del dataset de mensajes')
print ('Filas: {}'.format(msg_df.shape[0]))
print ('Columnas: {}'.format(msg_df.shape[1]))
print ('Cantidad de sesiones: ', msg_df['session_id'].nunique())
print ('Cantidad de turnos: {}'.format(msg_df.shape[0]))
print ('Cantidad de turnos del tutor: {}'.format(len(msg_df[msg_df.sent_from=='tutor'])))
print ('Cantidad de turnos del estudiante: {}'.format(len(msg_df[msg_df.sent_from=='student'])))

# Caracterización del dataset de metadata
print ('\nDimensiones del dataset de metadata')
print ('Filas: {}'.format(md_df.shape[0]))
print ('Columnas: {}'.format(md_df.shape[1]))
print ('Cantidad de sesiones: ', md_df['session_id'].nunique())


Dimensiones del dataset de mensajes
Filas: 234375
Columnas: 6
Cantidad de sesiones:  3851
Cantidad de turnos: 234375
Cantidad de turnos del tutor: 120536
Cantidad de turnos del estudiante: 89706

Dimensiones del dataset de metadata
Filas: 63265
Columnas: 35
Cantidad de sesiones:  63265


## Cálculo de la longitud de los mensajes y features derivadas

In [ ]:
# Cálculo de las longitudes de los mensajes (cantidad de palabas)

msg_df['tokens'] = msg_df.text.apply(lambda x: literal_eval(x))
msg_df['n_words'] = msg_df.tokens.apply(lambda x: len(x))

In [ ]:
# Número de palabras tutor y estudiante

agg_tutor_msg_df = msg_df[msg_df['sent_from']=='tutor'].groupby('session_id').agg(
    avg_words_tutor = ('n_words', np.mean),
    n_words_tutor = ('n_words', sum),
    n_msg_tutor = ('sent_from', lambda x: x.eq('tutor').sum())
    
)
agg_student_msg_df = msg_df[msg_df['sent_from']=='student'].groupby('session_id').agg(
    avg_words_student = ('n_words', np.mean),
    n_words_student = ('n_words', sum),
    n_msg_student = ('sent_from', lambda x: x.eq('student').sum())
    
)

## Otras agregaciones del dataset de mensajes

In [ ]:
# Otras agregaciones del dataset de mensajes 
agg_msg_df = msg_df.groupby('session_id').agg(
        # started =('created_at', min),
        # ended =('created_at', max),
        duration = ('created_at', lambda x: (max(x) - min(x))),
        n_msg_bot = ('sent_from', lambda x: x.eq('bot').sum()),
        n_msg_system = ('sent_from', lambda x: x.isin(list(['system alert', 'system info', 'system warn'])).sum()),
        n_msg_content_text = ('content_type', lambda x: x.eq('text').sum()),
        n_msg_content_image = ('content_type', lambda x: x.eq('image').sum()),
)

In [ ]:
print(agg_tutor_msg_df.shape)
print(agg_student_msg_df.shape)
print(agg_msg_df.shape)

(3338, 3)
(3767, 3)
(3851, 5)


## Unión de los dataframes

*A las métricas que no tienen valores en el left join se les asigna 0*

In [ ]:
# Joins
res_df = pd.merge(md_df, agg_msg_df, on=["session_id", "session_id"])
res_df = pd.merge(res_df, agg_tutor_msg_df, how="left", on=["session_id", "session_id"])
res_df = pd.merge(res_df, agg_student_msg_df, how="left", on=["session_id", "session_id"])

# Llenar los NAN producto del left join con ceros
res_df['avg_words_tutor'] = res_df['avg_words_tutor'].fillna(0)
res_df['n_words_tutor'] = res_df['n_words_tutor'].fillna(0)
res_df['n_msg_tutor'] = res_df['n_msg_tutor'].fillna(0)

res_df['avg_words_student'] = res_df['avg_words_student'].fillna(0)
res_df['n_words_student'] = res_df['n_words_student'].fillna(0)
res_df['n_msg_student'] = res_df['n_msg_student'].fillna(0)

## Caracterización de la variable `student_rating` a `student_rating_cat`

In [ ]:
# Categorización de la variable student_rating

def student_rating_category(x):
    
    if x <= 2:
        return 0
    
    if x >= 4:
        return 1

    return 'neutra'

# Eliminamos las filas nulas
res_df = res_df.dropna(subset=['student_rating'])

# Categorización de la variable numérica
res_df['student_rating_cat'] = res_df.student_rating.apply(student_rating_category)

# Remoción de las filas con calificación neutra
print ('Remoción de {} sesiones calificadas como neutras'.format(len(res_df[res_df['student_rating_cat'] == 'neutra'])))
res_df = res_df[res_df['student_rating_cat'] != 'neutra']

Remoción de 100 sesiones calificadas como neutras


In [ ]:
# Caracterización del dataset combinado
print ('\nDimensiones del dataset combinado')
print ('Filas: {}'.format(res_df.shape[0]))
print ('Columnas: {}'.format(res_df.shape[1]))
print ('Cantidad de sesiones: ', res_df['session_id'].nunique())


Dimensiones del dataset combinado
Filas: 2645
Columnas: 47
Cantidad de sesiones:  2645


## Drop de columnas que no se van a usar

In [ ]:
columns_to_keep = list()
columns_to_keep.append('tutor_id')
columns_to_keep.append('tutor_age')
columns_to_keep.append('session_tag_no_material')
columns_to_keep.append('session_tag_student_left')
columns_to_keep.append('session_tag_student_not_engaging')
columns_to_keep.append('student_complained')
columns_to_keep.append('student_complaint_clarity')
columns_to_keep.append('student_complaint_speed')
columns_to_keep.append('student_complaint_subject')
columns_to_keep.append('student_complaint_other')
columns_to_keep.append('session_tag_cheating')
columns_to_keep.append('session_tag_inappropriate')
columns_to_keep.append('session_tag_other_subject')
columns_to_keep.append('avg_words_tutor')
columns_to_keep.append('n_words_tutor')
columns_to_keep.append('n_msg_tutor')
columns_to_keep.append('avg_words_student')
columns_to_keep.append('n_words_student')
columns_to_keep.append('n_msg_student')
columns_to_keep.append('student_rating_cat')

final_df = res_df[columns_to_keep]

final_df = final_df.astype({'student_rating_cat':int})

mean_tutor_age = final_df.tutor_age.mean().astype(int)
for i in final_df[final_df.tutor_age.isna()].index:
    final_df.loc[i, ['tutor_age']] = mean_tutor_age

## Guardado del archivo de resultados

In [ ]:
# Salva archivo csv con el prefijo del archivo procesado
curated_data_file_name = msg_data_file_name.split('_')[0] + '_curated_data.csv'
curated_full_data_file_name = os.path.join(data_dir, curated_data_file_name)
final_df.to_csv(curated_full_data_file_name, index=False)

### Información del conjunto de datos generado

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2645 entries, 1 to 3850
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   tutor_id                          2645 non-null   Int64  
 1   tutor_age                         2645 non-null   Int64  
 2   session_tag_no_material           2645 non-null   Int64  
 3   session_tag_student_left          2645 non-null   Int64  
 4   session_tag_student_not_engaging  2645 non-null   Int64  
 5   student_complained                2645 non-null   Int64  
 6   student_complaint_clarity         2645 non-null   Int64  
 7   student_complaint_speed           2645 non-null   Int64  
 8   student_complaint_subject         2645 non-null   Int64  
 9   student_complaint_other           2645 non-null   Int64  
 10  session_tag_cheating              2645 non-null   Int64  
 11  session_tag_inappropriate         2645 non-null   Int64  
 12  sessio

### Descripción del conjunto de datos generado

In [ ]:
final_df.describe()

,tutor_id,tutor_age,session_tag_no_material,session_tag_student_left,session_tag_student_not_engaging,student_complained,student_complaint_clarity,student_complaint_speed,student_complaint_subject,student_complaint_other,session_tag_cheating,session_tag_inappropriate,session_tag_other_subject,avg_words_tutor,n_words_tutor,n_msg_tutor,avg_words_student,n_words_student,n_msg_student,student_rating_cat
count,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.0,2645.0,2645.0,2645.0,2645.0,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000,2645.000000
mean,141342.293384,37.148960,0.014745,0.084310,0.012476,0.0,0.0,0.0,0.0,0.0,0.010964,0.020794,0.006427,11.453993,424.447259,41.565217,4.065399,128.095652,30.545558,0.796219
std,152266.417651,13.773142,0.120552,0.277905,0.111020,0.0,0.0,0.0,0.0,0.0,0.104154,0.142721,0.079927,4.788981,469.201954,49.134930,1.967938,144.518904,34.049264,0.402884
min,6.000000,18.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34864.000000,23.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,8.577465,125.000000,11.000000,2.782609,36.000000,10.000000,1.000000
50%,53951.000000,33.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10.630435,281.000000,27.000000,3.818182,86.000000,21.000000,1.000000
75%,284699.000000,52.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,13.382812,564.000000,54.000000,5.000000,166.000000,39.000000,1.000000
max,507859.000000,63.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,59.000000,8062.000000,824.000000,23.000000,1679.000000,563.000000,1.000000
